In [1]:
import sys
sys.path.append('..')
from spyral.core.constants import QBRHO_2_P
from spyral.core.cluster import Cluster
from spyral.solvers.solver_interp_leastsq import Guess, interpolate_trajectory
from spyral.interpolate.track_interpolator import create_interpolator
from e20009_phases.InterpLeastSqSolverPhase import fit_model_interp
from spyral.core.run_stacks import form_run_string

from e20009_phases.config import  DetectorParameters
from spyral_utils.nuclear import NuclearDataMap
# from spyral.solvers.solver_interp_leastsq import solve_physics_interp as local_solver


import polars as pl
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib widget

from e20009_phases.InterpLeastSqSolverPhase import solve_physics_interp


In [2]:
det_params = DetectorParameters(
    magnetic_field=3.0,
    electric_field=60000.0,
    detector_length=1000.0,
    beam_region_radius=20.0,
    drift_velocity_path=Path(
        "C:\\Users\\schaeffe\\Desktop\\e20009-analysis\\e20009_parameters\\drift_velocity.csv"
    ),
    get_frequency=3.125,
    garfield_file_path=Path(
        "/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/e20009_efield_correction.txt"
    ),
    do_garfield_correction=False,
)

In [3]:


run_number = 297
event_number = 590868

In [4]:
## Proton Cluster for event  ##

cluster_index = 0
cluster_data = np.loadtxt("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10_d_p\\14_5_16_MeV_4_track\\297_590868_hand_clustered\\cluster0_proton.txt", delimiter=",")
cluster = Cluster(event_number, cluster_index, cluster_data)

# ## estimation_with_fake_input_wSQRTDEDX.ipynb ##

# proton_polar_guess = 66.74317123 * np.pi/180.0   # Radians
# proton_azimuthal_guess = 74.588245883  * np.pi/180.0   # Radians
# proton_brho_guess = 0.4392254615  # Tm
# proton_xvert_guess = 1.770714813  # mm
# proton_yvert_guess = -0.484343803  # mm
# proton_zvert_guess = 278.2503967   # mm
# guess = Guess(proton_polar_guess, proton_azimuthal_guess, proton_brho_guess, proton_xvert_guess, proton_yvert_guess, proton_zvert_guess)
# print(guess)

workspace_path= Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10_d_p\\14_5_16_MeV_4_track\\297_590868_hand_clustered")
estimate_file_path = workspace_path / f"{form_run_string(run_number)}.parquet"
estimate_df = pl.scan_parquet(estimate_file_path)
proton_cluster_index = 1
estimate_df = estimate_df.filter((pl.col("event") == event_number) & (pl.col("cluster_index") == proton_cluster_index)).collect()
print(estimate_df)
guess = Guess(estimate_df['polar'][0], estimate_df['azimuthal'][0], estimate_df['brho'][0], estimate_df['vertex_x'][0], estimate_df['vertex_y'][0], estimate_df['vertex_z'][0])
print(guess)


shape: (1, 23)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ event  ┆ cluster_in ┆ cluster_la ┆ ic_amplit ┆ … ┆ sqrt_dEdx ┆ dE        ┆ arclength ┆ direction │
│ ---    ┆ dex        ┆ bel        ┆ ude       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ i64    ┆ ---        ┆ ---        ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ i64       │
│        ┆ i64        ┆ i64        ┆ f64       ┆   ┆           ┆           ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 590868 ┆ 1          ┆ 1          ┆ 945.0     ┆ … ┆ 34.60952  ┆ 692443.99 ┆ 578.08738 ┆ 0         │
│        ┆            ┆            ┆           ┆   ┆           ┆ 9932      ┆ 7         ┆           │
└────────┴────────────┴────────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘
Guess(polar=0.3201211831031183, azimuthal=2.3192568451913127, brho=0.7328089

In [5]:
## Triton Cluster ## 

# cluster_index = 3
# cluster_data = np.loadtxt("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\B10(d,p)\\14_5_16_MeV_4_track\\256_597527\\cluster3.txt", delimiter=",")
# cluster = Cluster(event_number, cluster_index, cluster_data)

# ## estimation_with_fake_input_wSQRTDEDX.ipynb ##

# triton_polar_guess = 15.73605101 * np.pi/180.0   # Radians
# triton_azimuthal_guess = 320.899109 * np.pi/180.0   # Radians
# triton_brho_guess = 1.187993438   # Tm
# triton_xvert_guess = -1.532697897  # mm
# triton_yvert_guess = -1.884500457  # mm
# triton_zvert_guess = 271.9177137  # mm

# guess = Guess(triton_polar_guess, triton_azimuthal_guess, triton_brho_guess, triton_xvert_guess, triton_yvert_guess, triton_zvert_guess)
# print(guess)

In [5]:
# Get drift velocity

dv_lf: pl.LazyFrame = pl.scan_csv(det_params.drift_velocity_path)
dv_df: pl.DataFrame = dv_lf.filter(pl.col("run") == run_number).collect()
mm_tb: float = dv_df.get_column("average_micromegas_tb")[0]
w_tb: float = dv_df.get_column("average_window_tb")[0]
mm_err: float = dv_df.get_column("average_micromegas_tb_error")[0]
w_err: float = dv_df.get_column("average_window_tb_error")[0]

print(dv_df)

shape: (1, 5)
┌─────┬───────────────────────┬────────────────────────┬───────────────────┬───────────────────────┐
│ run ┆ average_micromegas_tb ┆ average_micromegas_tb_ ┆ average_window_tb ┆ average_window_tb_err │
│ --- ┆ ---                   ┆ error                  ┆ ---               ┆ or                    │
│ i64 ┆ f64                   ┆ ---                    ┆ f64               ┆ ---                   │
│     ┆                       ┆ f64                    ┆                   ┆ f64                   │
╞═════╪═══════════════════════╪════════════════════════╪═══════════════════╪═══════════════════════╡
│ 297 ┆ 60.866667             ┆ 0.530712               ┆ 390.5             ┆ 0.923353              │
└─────┴───────────────────────┴────────────────────────┴───────────────────┴───────────────────────┘


In [6]:
z = 1 # atomic number

a_p = 1
a_t = 3 # mass number
nuclear_map = NuclearDataMap()
proton = nuclear_map.get_data(z, a_p)
# triton = nuclear_map.get_data(z, a_t)
# print(nucleus.A)

In [7]:
        # Result storage
phys_results: dict[str, list] = {
            "event": [],
            "cluster_index": [],
            "cluster_label": [],
            "vertex_x": [],
            "sigma_vx": [],
            "vertex_y": [],
            "sigma_vy": [],
            "vertex_z": [],
            "sigma_vz": [],
            "brho": [],
            "sigma_brho": [],
            "ke": [],
            "sigma_ke": [],
            "polar": [],
            "sigma_polar": [],
            "azimuthal": [],
            "sigma_azimuthal": [],
            "redchisq": [],
        }

In [10]:
path_to_ODE_solution_mesh = Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\InterpSolver_assets\\1H_in_2H2_600Torr.npy")    # Mesh for protons 
# path_to_ODE_solution_mesh = Path("c:\\Users\\schaeffe\\Desktop\\e20009_analysis-output\\InterpSolver_assets\\3H_in_2H2_600Torr.npy")    # Mesh for tritons
tracks = create_interpolator(path_to_ODE_solution_mesh)

In [11]:

print(tracks.polar_bins)

360


In [12]:
solve_physics_interp(
    run_number,
    event_number,
    proton_cluster_index,
    cluster,
    guess,
    proton,
    tracks,
    det_params,
    w_tb,
    mm_tb,
    w_err,
    mm_err,
    phys_results,
)

# Write out the results
physics_df = pl.DataFrame(phys_results)
output_file_path = workspace_path / f"run_{run_number:04d}_{proton.isotopic_symbol}.parquet"
physics_df.write_parquet(output_file_path)

In [12]:
result = fit_model_interp(cluster, guess, proton, tracks, det_params, w_tb, mm_tb, w_err, mm_err)
if result is None:
    print('Guess outside of interpolation range!')
best_fit_trajectory = interpolate_trajectory(result, tracks, proton)
cluster.data[:, :3] *= 0.001



[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 1005
    # data points      = 36
    # variables        = 6
    chi-square         = 7.10650011
    reduced chi-square = 0.23688334
    Akaike info crit   = -46.4103263
    Bayesian info crit = -36.9092127
[[Variables]]
    brho:        0.67473110 +/- 0.00856385 (1.27%) (init = 0.7328089)
    polar:       0.76528515 +/- 0.01280462 (1.67%) (init = 0.3201212)
    vertex_rho:  0.01789121 +/- 0.00336847 (18.83%) (init = 0.0006494931)
    vertex_phi:  3.29075311 +/- 0.14736372 (4.48%) (init = 3.891981)
    vertex_x:   -0.01769255 +/- 0.00347678 (19.65%) == 'vertex_rho * cos(vertex_phi)'
    vertex_y:   -0.00265878 +/- 0.00249194 (93.73%) == 'vertex_rho * sin(vertex_phi)'
    vertex_z:    0.39698840 +/- 0.00506381 (1.28%) (init = 0.3895799)
    azimuthal:   4.69386914 +/- 0.01355478 (0.29%) (init = 2.319257)
[[Correlations]] (unreported correlations are < 0.100)
    C(vertex_rho, azimuthal)  = +0.7907
    C(brho, ve

In [13]:
proton_polar_final = result["polar"].value * 180 / np.pi
proton_azimuthal_final = result["azimuthal"].value
proton_brho_final = result["brho"].value
proton_xvert_final = result["vertex_x"].value
proton_yvert_final = result["vertex_y"].value
proton_zvert_final = result["vertex_z"].value

momentum = proton_brho_final  * QBRHO_2_P
kinetic_energy = np.sqrt(momentum**2.0 + proton.mass**2.0) - proton.mass
print(kinetic_energy)
print(proton_polar_final)

21.556763895621657
43.847609317510205


In [ ]:
# triton_polar_final = result["polar"].value * 180 / np.pi
# triton_azimuthal_final = result["azimuthal"].value
# triton_brho_final = result["brho"].value
# triton_xvert_final = result["vertex_x"].value
# triton_yvert_final = result["vertex_y"].value
# triton_zvert_final = result["vertex_z"].value

# momentum = triton_brho_final  * QBRHO_2_P
# kinetic_energy = np.sqrt(momentum**2.0 + triton.mass**2.0) - triton.mass
# print(kinetic_energy)
# print(triton_polar_final)

In [ ]:
# Plot fit
fig = make_subplots(2, 2, subplot_titles=["XY Projection", "XZ Projection", "YZ Projection"], specs=[[{"rowspan": 2}, {}],[None, {}]])
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 0],
        y=cluster.data[:, 1],
        mode="markers", 
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 0],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_x"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_x"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False,
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False
    ),
    row=2,
    col=2
)

fig["layout"]["xaxis1"]["title"] = "X (m)"
fig["layout"]["xaxis1"]["range"] = [-0.3, 0.3]
fig["layout"]["yaxis1"]["title"] = "Y (m)"
fig["layout"]["yaxis1"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis2"]["title"] = "Z (m)"
fig["layout"]["xaxis2"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis2"]["title"] = "X (m)"
fig["layout"]["yaxis2"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis3"]["title"] = "Z (m)"
fig["layout"]["xaxis3"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis3"]["title"] = "Y (m)"
fig["layout"]["yaxis3"]["range"] = [-0.3, 0.3]

fig.update_layout(
    width=1450,
    height=725
)